# Verificação formal de programas

A *verificação formal de programas* tem por objectivo garantir que um programa satisfaz a sua especificação.
Uma *especificação* é um modelo dum sistema que contém uma descrição do seu comportamento desejado ("*o que*" deve ser implementado, por oposição a "*como*"). Um *programa* é uma implementação concreta do sistema.

## A metodologia de Floyd-Hoare 

A metodologia de Floyd-Hoare estabelece uma lógica de programas em que as fórmulas são da forma
$\{\phi\} S \{\psi\}$, sendo $\phi$ e $\psi$ predicados e $S$ um programa. $\phi$ representa as *pré-condições* (condições que se assumem estarem garantidas quando o programa inicia a sua execução) e $\psi$ representa as *pós-condições* (condições que têm que ser asseguradas quando o programa termina). Chama-se especificação ao par $(\phi,\psi)$ e *triplo de Hoare* à fórmula $\{\phi\} S \{\psi\}$.

A verificação formal de programas permite garantir que um programa se comporta de acordo com a sua especificação, seguindo a seguinte estratégia. Numa primeira fase (a *geração de condições de verificação*) transforma-se o programa e a sua especificação numa fórmula lógica, chamada *condição de verificação* (VC), que se for válida garante que o programa satisfaz a especificação. Numa segunda fase é usado um *SMT solver* para determinar a validade da VC (ou encontrar modelos que descrevem um traço do programa que conduz à violação da propriedade).
É assim uma técnica de análise estática de programas que permite detectar erros que, eventualmente, só seriam detectados em tempo de execução.


Vamos considerar uma linguagem imperativa simples, de variáveis de tipo inteiro, em que um programa não é mais do que uma sequência dos seguintes comandos e anotações:

`skip` | `x = e` | `if b then C1 then C2` | `while b do C` | `assume P`| `assert P`

A metodologia que vamos aplicar baseia-se na utilização de uma linguagem intermédia, que permita representação do programa e da sua especificação, e a partir da qual é fácil gerar a denotação lógica do programa.
Assim, o primeiro passo consiste na tradução do programa e da especificação para essa linguagem intermédia, e o segundo passo na transformação dessa representação do programa numa VC. 

A noção de *fluxo de programa* é fundamental para a caracterização de programas imperativos e está na base da linguagem intermédia que vamos usar e a que chamaremos *linguagem de fluxos*. 
Um fluxo é basicamente uma sequência gerada com base nas seguintes construções:
`skip` que corresponde à sequência vazia, `;` que acrescenta um comando a um fluxo, e `||` que permite criar um fluxo que corresponde a uma escolha não-determinista.

A construção `(Fluxo || Fluxo)` denota escolha não-determinista entre dois fluxos. 
Tipicamente estes fluxos estão encabeçados por um `assume P` e somente quando `P` for verdadeiro é que esse fluxo está qualificado para execução. 
Esta construção é usada na tradução da instrução `if b then C1 else C2` que é reescrita da seguinte forma:
`(assume b ; C1 || assume (not b) ; C2)`.

A tradução da instrução `while b do C` é mais complexa e será analizada mais abaixo.

Um triplo de Hoare $\{\phi\} S \{\psi\}$ pode ser representado por:  ${\sf assume }\: \phi\; ; S \; ; {\sf assert } \:\psi$.

Para a geração das VCs há duas técnicas standard:

- *Weakest pre-condition* (WPC): onde, dada a representação do programa e a pós-condição, é gerada a pré-condição mais fraca que é suficiente para garantir que quando o programa termina a pós-condição é assegurada.

- *Strongest post-condition* (SPC): onde, dada a representação do programa e a pré-condição, é gerada a pós-condição mais forte que é possível garantir quando o programa termina.



### Weakest pre-condition

A denotação `[C]` associa a cada fluxo `C` um predicado que  caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica WPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = True \\
[{\sf assert}\:\phi] = \phi \\
[ x = e ] = True \\
[(C_1 || C_2)] = [C_1] \wedge [C_2] \\
\\
[{\sf skip}\, ; C] = [C] \\
[{\sf assume}\:\phi\, ; C] = \phi \to [C] \\
[{\sf assert}\:\phi\, ; C] = \phi \wedge [C] \\
[ x = e \, ; C] = [C][e/x] \\
[(C_1 || C_2)\, ; C] = [(C_1 ; C) || (C_2 ; C)]
\end{array}
$


Começamos por importar as bibliotecas do PySMT e definir a função `prove` que verifica a validade de uma fórmula lógica usando um SMT solver.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
        else:
            print("Proved.")

### Exercício 1

Considere o seguinte programa anotado que faz o *swap* de duas variáveis usando operações aritméticas.
```python
assume x == a and y == b;
x = x + y;
y = x - y;
x = x - y;
assert x == b and y == a
```
Usando a abordagem WPC calcule a denotação lógica deste programa (a sua condição de verificação).

```python
[assume x == a and y == b; x = x + y; y = x - y; x = x - y; assert x == b and y == a]
=
(completar)
```

### Exercício 2

Use a função `prove` para verificar a condição de verificação obtida no exercício 1, assumindo que as variáveis são inteiros de tamanho ilimitado. Pode utilizar a função `substitute(a,{b:c})` para substituir em `a` todas as ocorrências de `b` por `c`.

In [ ]:
# completar

### Exercício 3

Considere o seguinte programa anotado que calcula o máximo de dois números.
```python
if x>y 
   then m = x
   else m = y;
assert m >= x and m >= y and (m == x or m == y)
```
Usando a abordagem WPC calcule a denotação lógica deste programa (a sua condição de verificação). Note que primeiro tem que traduzir o programa para a linguagem de fluxos. Use também o solver para verificar a sua validade.

Programa de fluxos
```python
       (completar)
```

Denotação lógica do programa via WPC
```python
Seja   pos =  m >= x and m >= y and (m == x or m == y)
```
```python
       (completar)
```

In [ ]:
# completar

### Strongest post-condition

Na abordagem SPC a denotação de um fluxo com um comando de atribuição introduz um quantificador existencial, o que não é adequado à verificação com SMT solvers: 
$ \quad [ C \; ; x = e ] \; =  \; \exists a. (x = e[a/x]) \wedge [C][a/x] $

Para lidar com este problema pode-se converter o programa original ao formato "*single assignment*" (SA).
Num programa SA cada variável só pode ser usada depois de ser atribuida e só pode ser atribuída uma única vez.

Um programa (onde variáveis são atribuídas mais do que uma vez) pode ser reescrito num programa SA criando "clones" distintos das variáveis de forma a que seja possível fazer uma atribuição única a cada instância.

Neste caso, a denotação `[C]` associa a cada fluxo `C` um predicado que caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica SPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = \phi \\
[{\sf assert}\:\phi] = \phi \\
[x = e ] = (x = e)\\
[(C_1 || C_2)] = [C_1] \vee [C_2] \\
\\
[C \, ; {\sf skip}\;] = [C] \\
[C \, ;{\sf assume}\:\phi] = [C] \wedge \phi \\
[C \, ;{\sf assert}\:\phi] = [C] \to \phi \\
[ C \, ; x = e ] = [C] \wedge (x = e)\\
[C\,; (C_1 || C_2)] = [(C ; C_1) || (C; C_2)]
\end{array}
$



### Exercício 4

Usando a abordagem SPC calcule a denotação lógica do programa que faz o *swap* de duas variáveis (a sua condição de verificação). Note que primeiro tem que traduzir o programa para a linguagem de fluxos SA. Use também o SMT solver para verificar a validade da condição de verificação obtida.

```python
[assume x == a and y == b; x0 = x + y; y0 = x0 - y; x1 = x0 - y0; assert x1 == b and y0 == a]
=


(completar)
```

In [ ]:
# completar

### Exercício 5

Usando a abordagem SPC calcule a denotação lógica do programa (a sua condição de verificação) que calcula o máximo de 2 números. Note que primeiro tem que traduzir o programa para a linguagem de fluxos SA. Use também o SMT solver para verificar a validade da condição de verificação obtida.

```python
(completar)
```

In [ ]:
# completar

## Programas iterativos

Na verificação de programas com ciclos a dificuldade está na verificação da validade do triplo de Hoare 
$\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}$.

Uma primeira abordagem assenta na noção de *invariante*, uma propriedade que tem que ser válida à entrada do ciclo, ao final de cada iteração, e que deve ser suficientemente forte para à saida do ciclo garantir a pós-condição.

Esta ideia está sintetizada na regra:
$$
\frac{\{\phi\}{\sf skip} \{\theta\} \quad \{\theta \wedge b\} C \{\theta\} \quad \{\theta \wedge \neg b\}{\sf skip} \{\psi\} }{\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}}
$$

Assim, para garantir a validade de $\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}$ basta garantir a validade das premissas desta regra, gerando as VCs de cada um desses triplos (que correspondem à "inicialização", "preservação" e "utilidade"). Se as VCs forem válidas, então o triplo que está na conclusão é válido.

A principal dificuldade neste processo está na descoberta do invariante $\theta$ apropriado.

### Exercício 6

Considere o seguinte programa anotado (incluindo o invariante de ciclo) que calcula o máximo de um array de inteiros.

```python
assume n >= 1 and i == 1 and m == A[0]
while i<n:
    invariant i<=n and forall j . 0 <= j < i -> m >= A[j]
    if A[i]>m:
        m = A[i]
    i = i+1
assert forall j . 0 <= j < n -> m >= A[j]
```
Indique os fluxos correspondentes aos triplos de Hoare necessários para verificar a inicialização, preservação e utilidade deste invariante. Calcule também as respectivas condições de verificação.

```python
Sejam  pre = n >= 1 and i == 1 and m == A[0]
       pos = forall j . 0 <= j < n -> m >= A[j]
       inv = i<=n and forall j . 0 <= j < i -> m >= A[j]
```

Denotações via WPC
```python
init = ...
util = ...
pres = ...
```

### Exercício 7

Utilize um SMT solver para verificar a validade das condições de verificação obtidas no exercício 6. 
Pode usar a instrução `Symbol('A', ArrayType(INT, INT))` para declarar a variável lógica do tipo array de inteiros para inteiros e `Select(A,i)` para recuperar o valor que está na posição `i` do array `A`.


In [ ]:
A = Symbol('A', ArrayType(INT, INT))
i = Symbol('i',INT)
m = Symbol('m',INT)
n = Symbol('n',INT)
j = Symbol('j',INT)


pre = And(n>=Int(1),Equals(i,Int(1)),Equals(m,Select(A,Int(0))))
pos = ForAll([j],Implies(And(j>=Int(0),j<n),m >= Select(A,j)))
inv = And(i<=n,ForAll([j],Implies(And(j>=Int(0),j<i),m >= Select(A,j))))

# completar

init = ...
util = ...
pres = ...


prove(init)
prove(util)
prove(pres)

## Metodologia havoc

Uma metodologia alternativa consiste em representar o programa original (com ciclos) na linguagem intermédia de fluxos (sem ciclos), substituindo cada ciclo ${\sf while} \; b \;{\sf do } \;C$ por um fluxo $H$ tal que:
se $\{\phi\} H \{\psi\}$ se verificar, então $\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}$ verifica-se também.

O comando `havoc x` pode ser descrito informalmente como uma atribuição a `x` de um valor arbitrário.  Em termos de denotação lógica usando a técnica WPC teremos 

$$
[{\sf havoc}\; x \; ; C] = \forall x. \,[C]
$$

Na metodologia *havoc*, o ciclo (${\sf while} \; b \;{\sf do }\{\theta\} \;C$), com anotação de invariante $\theta$ é transformado num fluxo não iterativo da seguinte forma

$$
{{\sf assert}\; \theta\; ; \sf havoc }\;\vec{x} \; ; (\,({\sf assume }\; b \wedge \theta \; ; \; C \; ; {\sf assert}\;\theta \; ; {\sf assume}\; \mathit{False}) \: || \:
{\sf assume}\; \neg b \wedge \theta \,)
$$

onde $\vec{x}$ representa as *variáveis atribuídas em $C$*.

Observe como a denotação do triplo de Hoare $\{\phi\} {\sf while} \; b \;{\sf do}\{\theta\}\,C \,\{\psi\}$,  traduzido desta forma,
permite garantir as propriedades de "inicialização", "preservação" e "utilidade" do invariante $\theta$

$$
\begin{array}{l}
[\,{\sf assume}\;\phi\; ;{{\sf assert}\; \theta\; ; \sf havoc }\;\vec{x} \; ; (\,({\sf assume }\; b \wedge \theta \; ; \; C \; ; {\sf assert}\;\theta \; ; {\sf assume}\; \mathit{False}) \: || \:
{\sf assume}\; \neg b \wedge \theta \,)\; ; {\sf assert} \; \psi \,] \\ = \\
\phi \to \theta \wedge \forall \vec{x}. \, (\,(b \wedge \theta \to [C\;; {\sf assert}\; \theta ]) \wedge (\neg b \wedge \theta \to \psi )\,)
\end{array}
$$

Note que $[ \,{\sf assume}\; \mathit{False}\;; {\sf assert} \; \psi \,] = \mathit{False} \to \psi = \mathit{True}$.

### Codificação de funções lógicas

Considere o seguinte programa `FACT` para o cálculo do factorial de um número. O programa está já anotado com pré-condição, pós-condição  e invariante de ciclo que deverá premitir  provar a sua correção face a esse contrato.

```python
assume n >= 0;
f = 1; i = 1;
while (i <= n):
    invariante 0 < i and i <= n+1 and f == (i-1)!  
    f = f*i;
    i = i+1
assert f == (n)!    
```

Nas anotações lógicas recorremos à função matemática $(n)!$ que denota o factorial de $n$.
Para poder usar um *SMT solver* para o ajudar a verificar a correção do programa face à sua especificação, teremos acrescentar à teoria do *SMT solver* a codificação lógica da função factorial, uma vez que a teoria de inteiros não vem munida de tal função.

Uma forma de o fazer é definir na teoria de inteiros uma função lógica ${\sf fact}$ cuja semântica é dada por um conjunto de axiomas. Por exemplo, assim:

$$
\begin{array}{l}
{\sf fact} (0) = 1 \\
\forall n . \, n > 0 \to {\sf fact}(n) = n * {\sf fact}(n-1)
\end{array}
$$

Usando o pySMT isso pode ser feito da seguinte forma:

In [ ]:
fact = Symbol('fact', FunctionType(INT,[INT]))
n = Symbol('n',INT)

ax1 = Equals(fact(Int(0)),Int(1))
ax2 = ForAll([n],Implies(n>Int(0),Equals(fact(n),n*fact(n-Int(1)))))
axioms = And(ax1,ax2)

Podemos agora testar a correção desta axiomatização verificando que o valor de $\mathsf{fact}(5) = 120$.

In [ ]:
prove(Implies(axioms,Equals(fact(Int(5)),Int(120))))

### Exercício 8

Aplique agora a metodologia *havoc* para provar a correcção do programa `FACT` face à especificação.
Sugestão: começe por traduzir o programa para a linguagem de fluxos e construa a VC correspondenta à sua denotação lógica usando a técnica da WPC. Tenha em atenção que as substituições que resultam das atribuições `f=1; i=1;` não terão impacto na parte da formula quantificada com $\forall f. \forall i. \ldots$  uma vez que as variáveis $f$ e $i$ estão ligadas.

Programa de fluxos.

```python
Sejam  pre = n >= 0
       inv = 0 < i and i <= n+1 and f == (i-1)! 
       pos = f == (n)! 

(completar)
````

Denotação lógica com WPC.

```python
(completar)
```

In [ ]:
# completar